In [1]:
import numpy as np
import torch
import torchvision
import cv2
import matplotlib.pyplot as plt
import  torchvision.transforms as transforms
import timm
import json

In [2]:
with open('classes.json', 'r') as f:
    classes = json.load(f)

In [11]:
model = timm.create_model(model_name='mobilenetv3_large_100', pretrained=True)
model.eval()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to /Users/19065443/.cache/torch/hub/checkpoints/mobilenetv3_large_100_ra-f55367f5.pth


MobileNetV3(
  (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): Hardswish()
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (se): Identity()
        (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [12]:
# model = torchvision.models.mobilenet_v3_small(pretrained=False, progress=False, )
# model.state_dict = torch.load('../model/mobilenet_v3_small.pth')
# model.eval()

In [13]:
batch_size = 1
channels = 3
height = 1024
width = 1024
# image = cv2.imread('../../../images/swin.jpeg')
image = cv2.imread('../../../images/n01491361_tiger_shark.jpeg')
resized_image = cv2.resize(image, (height, width))


In [14]:
transform = transforms.ToTensor()
prediction = torch.argmax(model.float().forward(torch.unsqueeze(transform(resized_image), 0)).data).numpy()
classes[prediction]

'toilet seat'

In [15]:
transform = transforms.ToTensor()
prediction = torch.argmax(model.double().forward(torch.unsqueeze(transform(resized_image / 255.), 0)).data).numpy()
classes[prediction]

'toilet seat'

In [16]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])
prediction = torch.argmax(model.double().forward(torch.unsqueeze(transform(resized_image / 255.), 0)).data).numpy()
classes[prediction]

'tiger shark'

In [12]:
transform(resized_image)

tensor([[[ 0.1426,  0.1426,  0.1426,  ..., -0.1486, -0.1314, -0.1143],
         [ 0.1426,  0.1426,  0.1426,  ..., -0.1314, -0.1314, -0.1143],
         [ 0.1426,  0.1426,  0.1426,  ..., -0.1486, -0.1486, -0.1314],
         ...,
         [-0.3883, -0.3883, -0.3883,  ..., -0.6794, -0.6623, -0.6452],
         [-0.3883, -0.3883, -0.3883,  ..., -0.6794, -0.6452, -0.6281],
         [-0.3883, -0.3883, -0.3883,  ..., -0.6794, -0.6452, -0.6281]],

        [[-0.1975, -0.1975, -0.1975,  ..., -0.5651, -0.5476, -0.5301],
         [-0.1975, -0.1975, -0.1975,  ..., -0.5476, -0.5476, -0.5301],
         [-0.1975, -0.1975, -0.1975,  ..., -0.5651, -0.5651, -0.5476],
         ...,
         [-0.8452, -0.8452, -0.8277,  ..., -0.8277, -0.7927, -0.7752],
         [-0.8452, -0.8452, -0.8277,  ..., -0.8277, -0.7752, -0.7577],
         [-0.8452, -0.8452, -0.8277,  ..., -0.8277, -0.7752, -0.7577]],

        [[-0.3404, -0.3404, -0.3404,  ..., -0.9156, -0.8981, -0.8807],
         [-0.3404, -0.3404, -0.3404,  ..., -0

In [11]:
dummy_input = torch.randn(1, 3, 1024, 1024)
torch.onnx.export(model, dummy_input, 'efficientnet_b4.onnx', export_params=True)

In [12]:
import onnx
onnx_model = onnx.load('efficientnet_b4.onnx')
onnx.checker.check_model(onnx_model)

In [ ]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("efficientnet_b4.onnx")
